In [136]:
import pandas as pd
import csv

In [137]:
with open("checkins.dat", "r") as f:
    with open("checkins.csv", "wb") as f1:
        cw = csv.writer(f1)
        line = f.readline()
        while line:
            s = map(lambda x: x.strip(), line.split('|'))
            cw.writerow(s)
            line = f.readline()

In [139]:
data = pd.read_csv("checkins.csv", sep=",", dtype=str)

In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021969 entries, 0 to 1021968
Data columns (total 6 columns):
id            1021968 non-null object
user_id       1021966 non-null object
venue_id      1021966 non-null object
latitude      396634 non-null object
longitude     396634 non-null object
created_at    1021966 non-null object
dtypes: object(6)
memory usage: 46.8+ MB


In [141]:
data = data[~data.latitude.isnull()]

In [142]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 2 to 1021965
Data columns (total 6 columns):
id            396634 non-null object
user_id       396634 non-null object
venue_id      396634 non-null object
latitude      396634 non-null object
longitude     396634 non-null object
created_at    396634 non-null object
dtypes: object(6)
memory usage: 21.2+ MB


In [143]:
from sklearn.cluster import MeanShift

In [144]:
X = data[:100000][["latitude", "longitude"]]

In [145]:
X = X.astype(float)

In [146]:
cls = MeanShift(bandwidth=0.1)
cls.fit(X)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [149]:
np.unique(cls.labels_).shape

(3229L,)

In [161]:
import numpy as np
labels = np.unique(cls.labels_)
labels_counts = np.zeros(labels.shape)
labels_l = np.zeros(labels.shape)
for i, label in enumerate(labels):
    labels_counts[i] = np.sum(cls.labels_==label)
    labels_l[i] = label
#labels_counts.sort(key = lambda x: -x)

In [162]:
labels_cf = labels_counts[labels_counts>15]
labels_lf = labels_l[labels_counts>15]

In [157]:
longitudes = [33.751277, 25.867736, 51.503016, 52.378894, 39.366487, -33.868457]
latitudes = [-118.188740, -80.324116, -0.075479, 4.885084, 117.036146, 151.205134]

In [170]:
dists = []
for l,c in zip(labels_lf, labels_cf):
    dists_t = []
    for longit, latit in zip(longitudes, latitudes):
        v = cls.cluster_centers_[l] - np.array([longit, latit])
        dists_t.append(np.dot(v, v))
    dists.append((l,np.min(dists_t)))

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [173]:
dists.sort(key = lambda x: x[1])

In [174]:
dists

[(413.0, 6.1383435474385181e-05),
 (375.0, 8.7484523675144147e-05),
 (405.0, 0.00051411327615484234),
 (58.0, 0.0025058328805651507),
 (51.0, 0.0050194011900760883),
 (29.0, 0.017985232829248871),
 (166.0, 0.028024756866587088),
 (92.0, 0.035674128493271909),
 (87.0, 0.038329595578678682),
 (42.0, 0.044863703509826942),
 (285.0, 0.04938763550948061),
 (315.0, 0.073604102108897881),
 (119.0, 0.087012543658036762),
 (55.0, 0.091367224654354179),
 (27.0, 0.092860679506418964),
 (11.0, 0.099151801416538038),
 (32.0, 0.11479253475179307),
 (158.0, 0.11617575938854),
 (17.0, 0.1434042236031941),
 (47.0, 0.14954170433211428),
 (64.0, 0.15902391454044734),
 (24.0, 0.16808605549613967),
 (35.0, 0.16844288870139407),
 (50.0, 0.20075356588691057),
 (100.0, 0.24343117066513756),
 (144.0, 0.29210838600052624),
 (90.0, 0.3297109321954248),
 (75.0, 0.36372059477570851),
 (223.0, 0.38324777769610308),
 (284.0, 0.40772585242425124),
 (286.0, 0.45030658732953066),
 (396.0, 0.45045432222536042),
 (143.0,

In [177]:
dists[0][0]

413.0

In [179]:
cls.cluster_centers_[int(dists[0][0])]

array([ -33.86063043,  151.20477593])

In [182]:
coords = cls.cluster_centers_[int(dists[0][0])]
coords = map(str, coords)
with open("answer.txt", "w") as f:
    f.write(" ".join(coords))
    